# Logging experiments and managing models with mlflow

## First:
- open a new terminal window.
- run 
`conda activate workshop && mkdir mlflow-server && cd mlflow-server && mlflow server -h 0.0.0.0 --backend-store-uri sqlite:///mlflow.db  --default-artifact-root ./artifacts`

In [10]:
from training_code import download_data, split_data, evaluate_model
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

import mlflow

In [18]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mlflow-demo")

INFO: 'mlflow-demo' does not exist. Creating a new experiment


In [12]:
download_data("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv","data/winequality-red.csv")

In [13]:
split_data("data/winequality-red.csv")

In [27]:
def train_model(x_train_path: str, y_train_path: str, params: dict):


    x_train = pd.read_csv(x_train_path)
    y_train = pd.read_csv(y_train_path)

    pipe = Pipeline([('scaler', StandardScaler()),
                     ('regressor', ElasticNet(alpha=params['alpha'],
                                              l1_ratio=params['l1_ratio'],
                                              random_state=42))])

    return pipe.fit(x_train, y_train)

In [28]:
params = {
    "alpha": 0.5,
    "l1_ratio": 0.5
}

In [37]:
with mlflow.start_run():
    mlflow.sklearn.autolog()
    model = train_model("data/x_train.csv","data/y_train.csv", params)

## Register model in UI

Open http://localhost:5000 and create a new model and model version based on the model of the experiment

## Use Model from model registry for prediction